# Design Under Uncertainty

This notebook was automatically generated from the Algorithms for Optimization source code. Each cell generates a figure from the original text. While this code is not optimized for use in lectures, we provide it here to be adapted for such projects. We hope you find it useful.

In [ ]:
include("support_code.jl");

In [ ]:
	using Distributions
	using Optim
	import QuadGK: quadgk

	p = let

		a = 1
		b = 2
		c = 4.5
		f = x -> 2.5 - 2exp(-100(x-a)^2) - exp(-(x-b)^2)  - 1.4exp(-0.15(x-c)^2)

		p = Plots.Plot[]
		x_arr = append!(collect(range(0.0,stop=1.5,length=300)), collect(range(1.5,stop=9,length=200)[2:end]))
		push!(p, Plots.Linear(x_arr, f.(x_arr), style="solid, thick, black, mark=none", legendentry="noise-free"))
		labels=["very low", "low", "high", "very high"]
		for (i,σ) in enumerate([0.1,0.5,1.0,2.0])
		    P = Normal(0,σ)
		    g = x -> quadgk(z->f(x+z)*pdf(P, z), -5, 5)[1]
		    push!(p, Plots.Linear(g, (0, 9), xbins=401, style="solid, pastelBlue!$(100-25(i-1)), mark=none", legendentry=labels[i]*" noise"))
		end

		min_a = optimize(f, a-0.5, a+0.5).minimizer
		min_b = optimize(f, b-0.5, b+0.5).minimizer
		min_c = optimize(f, c-1.0, c+1.0).minimizer

		Axis(p, width="12cm", height="6cm", style="axis lines=left, xtick={$(min_a), $(min_b), $(min_c)}, xticklabels={\$a\$, \$b\$, \$c\$},  xticklabel style={text height=2ex}, ytick=\\empty, legend style={draw=none, at={(0.5,-0.25)}, anchor=north, legend columns=-1, /tikz/every even column/.append style={column sep=0.5cm}}",
		    xlabel=L"x", ylabel="expected value", xmin=-0.5, ymin=-0.25)
    end
    plot(p)

In [ ]:
using Distributions
using Optim

p = let

	f = x -> x^3 + 1
	p = Plots.Plot[]
	push!(p, Plots.Linear(f, (0, 5), style="solid, black, mark=none"))

	ymin = -3

	δ = 0.49
	P = Normal(1.5,0.25)
	dom = quantile.(P, [0.5-δ, 0.5+δ])
	a, b = dom[1], dom[2]
	fa, fb = f(a), f(b)
	push!(p, Plots.Linear(x->2pdf(P, x)+ymin, (a, b), style="solid, pastelBlue, mark=none"))
	push!(p, Plots.Linear([-0.5, a, a], [fa, fa, ymin], style="draw=none, mark=none, name path=A, forget plot"))
	push!(p, Plots.Linear([-0.5, b, b], [fb, fb, ymin], style="draw=none, mark=none, name path=B, forget plot"))
	push!(p, Plots.Command("\\addplot[pastelBlue!40] fill between[of=A and B];"))
	_μ = (fa+fb)/2
	σ = optimize(σ->(quantile(Normal(_μ, σ), 0.5+δ) - fb)^2, 0.1, 10.0).minimizer
	Q = Normal(_μ, σ)
	y_arr = range(fa, stop=fb, length=101)
	x_arr = [3*(pdf(Q, y)-pdf(Q,fa))-0.5 for y in y_arr]
	push!(p, Plots.Linear(x_arr, y_arr, style="solid, pastelBlue, mark=none"))
	push!(p, Plots.Node("robust",0,fb, style="above right"))

	P = Normal(4,0.25)
	dom = quantile.(P, [0.5-δ, 0.5+δ])
	a, b = dom[1], dom[2]
	fa, fb = f(a), f(b)
	push!(p, Plots.Linear(x->2pdf(P, x)+ymin, (a, b), style="solid, pastelBlue, mark=none"))
	push!(p, Plots.Linear([-0.5, a, a], [fa, fa, -3], style="draw=none, mark=none, name path=A, forget plot"))
	push!(p, Plots.Linear([-0.5, b, b], [fb, fb, -3], style="draw=none, mark=none, name path=B, forget plot"))
	push!(p, Plots.Command("\\addplot[pastelBlue!40] fill between[of=A and B];"))
	_μ = (fa+fb)/2
	σ = optimize(σ->(quantile(Normal(_μ, σ), 0.5+δ) - fb)^2, 0.1, 10.0).minimizer
	Q = Normal(_μ, σ)
	y_arr = range(fa, stop=fb, length=101)
	x_arr = [3*(pdf(Q, y)-pdf(Q,fa))-0.5 for y in y_arr]
	push!(p, Plots.Linear(x_arr, y_arr, style="solid, pastelBlue, mark=none"))
	push!(p, Plots.Node("sensitive",0,fb, style="above right"))

	Axis(p, width="9cm", xmin=-0.5, ymin=ymin, xlabel=L"x", ylabel=L"y", style="axis lines=left, axis on top, xtick=\\empty, ytick=\\empty")
end
plot(p)